In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import progressbar

from collections import deque

In [64]:
class backtest_sys(object):
    def __init__(self, data_path, max_look_back, split_price=600, commodity_spot=200, initial_cash=1e6,max_marketposition=6):
        data = pd.read_csv(data_path, sep=',')
        self.data_date = data['Date'].values
        self.data_high = data['High'].values
        self.data_low = data['Low'].values
        self.data_close = data['Close'].values
        self.data_open = data['Open'].values
        self.data_vol = data['TotalVolume'].values
        self.split_price = split_price
        self.commodity_spot = commodity_spot
        self.initial_cash = initial_cash 
        self.data_len = len(data)
        self.max_look_back = max_look_back
        #加入最大倉位的概念
        self.max_marketposition = max_marketposition
        self.__reset()  

    def __reset(self):
        self.step_counter = 0
        self.cash = self.initial_cash
        self.total_value = self.cash
        self.marketposition = 0
        self.openposition = 0
        #加入平倉價格的概念
        self.coverprice = 0
        self.marketposition_list = []
        #每筆交易使用的List,但最後不會使用到,會併入Total_trade_list中
        self.trading_list = []
        self.value_list = []
        #交易明細
        self.Total_trade_list = []
        #加入每次進出多少合約的概念
        self.contract = 0 
        #這邊的PnL是每日的權益變動
        self.PnL = 0
        self.PnL_list = []

    #把Long改成Buy,更接近Multichart   
    def __Buy(self,contract):
        #這邊先不用管,主要這個IF的意義在於不想買超過最大持倉數量,可尚未開發完成,可忽略
        if self.marketposition <= self.max_marketposition - contract :
            self.marketposition += contract
            self.cash -= self.split_price * contract
            self.cost = self.data_open[self.step_counter]
            #加入進場時間的概念
            self.time = self.data_date[self.step_counter]
            self.contract = contract
            #每次Operate後就開啟交易明細一次,運算用
            self.Total_trade_calculation()
         
    #把Short改成Sellshort,更接近Multichart          
    def __Sellshort(self,contract):
        #這邊先不用管,主要這個IF的意義在於不想買超過最大持倉數量,可尚未開發完成,可忽略
        if self.marketposition >= -(self.max_marketposition ) + contract : 
            self.marketposition -= contract
            self.cash -= self.split_price * contract
            self.cost = self.data_open[self.step_counter]
            #加入進場時間的概念
            self.time = self.data_date[self.step_counter]
            self.contract = contract
            #每次Operate後就開啟交易明細一次,運算用
            self.Total_trade_calculation()
            
    #把__Close改成__Sell和__Buytocover ,更接近Multichart,暫時想不到差異性,但我覺得MC這樣做一定有它的道理,我們使用過MC也更直觀可以寫策略
    def __Sell(self):
        self.cash -= self.split_price * self.contract
        #紀錄出場的價格
        self.coverprice = self.data_open[self.step_counter]
        #記錄出場的時間
        self.time = self.data_date[self.step_counter]
        self.profit = (self.coverprice - self.cost) * self.commodity_spot * self.contract
        self.cash += self.profit
        self.contract = 0    
        self.marketposition = 0
        #每次Operate後就開啟交易明細一次,運算用
        self.Total_trade_calculation()

    def __Buytocover(self):
        self.cash -= self.split_price * self.contract
        #紀錄出場的價格
        self.coverprice = self.data_open[self.step_counter] 
        #記錄出場的時間
        self.time = self.data_date[self.step_counter]
        self.profit = (self.cost - self.coverprice) * self.commodity_spot * self.contract
        self.cash += self.profit
        self.contract = 0
        self.marketposition = 0
        #每次Operate後就開啟交易明細一次,運算用
        self.Total_trade_calculation()
            
        
    def get_state(self):
        state = (self.data_date[self.step_counter], self.data_open[self.step_counter], self.data_high[self.step_counter],
                 self.data_low[self.step_counter], self.data_close[self.step_counter], self.data_vol[self.step_counter],
                 self.marketposition, self.openposition)
        return state



#加入交易明細,直接Call就能叫出來,以Dataframe方式呈現
    def Total_trade_calculation(self):
        if self.marketposition > 0:
            self.trading_list = []
            self.trading_list = [self.cost,self.time,"Long"]
        elif self.marketposition < 0:
            self.trading_list = []
            self.trading_list = [self.cost,self.time,"Short"]            
        elif self.marketposition == 0:
            self.trading_list.append(self.coverprice)
            self.trading_list.append(self.time)
            if self.trading_list[2] == "Long":
                self.trading_list.append(((self.coverprice - self.cost) * self.commodity_spot ) - self.split_price * 2 )
            else :
                self.trading_list.append(((self.cost - self.coverprice) * self.commodity_spot ) - self.split_price * 2 )
            self.Total_trade_list.append(self.trading_list)
        df = pd.DataFrame(env.Total_trade_list,columns =["Entry_price","Entry_time","Long/Short","Cover_price","Cover_time","PnL"])
        return df
    
#刪除Pass的部分(有待討論),主要原因是使用Pass容易造成有Bug會找不到,等到最後完成版在加入Pass或許更好一點        
    def step_operate(self, action,contract):
        if action == 'Buy':
            if self.marketposition == 0:
                self.__Buy(contract)
            #口數不限縮於 -1
            elif self.marketposition < 0: 
                self.__Buytocover()
                self.__Buy(contract)
                
        elif action == 'Sellshort':
            if self.marketposition == 0:
                self.__Sellshort(contract)
            #口數不限縮於 +1
            elif self.marketposition > 0:
                self.__Sell()
                self.__Sellshort(contract)
                
        elif action == 'Sell':
            if self.marketposition > 0:
                self.__Sell()
        elif action == 'Buytocover':
            if self.marketposition < 0:
                self.__Buytocover() 
        elif action == 'keep':
            pass
        
        
        
        self.marketposition_list.append(self.marketposition)
        
            
        if self.marketposition > 0: 
            self.openposition = (self.data_close[self.step_counter]-self.cost) * self.commodity_spot * self.contract
        elif self.marketposition < 0:
            self.openposition = (self.cost-self.data_close[self.step_counter]) * self.commodity_spot * self.contract
        else:
            self.openposition = 0
        #計算每日損益變動
        self.PnL = self.openposition + self.cash  - self.total_value   
        self.PnL_list.append(self.PnL)
        self.total_value = self.openposition + self.cash 
        self.value_list.append(self.total_value)
          
            
    def cal_info(self, start, end):
        self.xs = np.asarray([self.data_date[i] for i in range(self.max_look_back, self.data_len)])[start:end]
        self.open_plot = np.asarray([self.data_open[i] for i in range(self.max_look_back, self.data_len)])[start:end]
        self.long_position = [i-start  for i in range(start, end) if self.marketposition_list[i] > 0]
        self.short_position = [i-start for i in range(start, end) if self.marketposition_list[i] < 0]
        self.neutral_position = [i-start for i in range(start, end) if self.marketposition_list[i] == 0]
        self.drawdown_lis = []
        for i in range(len(self.value_list)):
            max_temp_equity = max(self.value_list[:i+1])
            drawdown = self.value_list[i] - max_temp_equity
            self.drawdown_lis.append(drawdown)   
    
#整合成較直觀的 DataFrame,下面有範例,直接call就能叫出來
    def calculation_DataFrame(self):    
        df = pd.DataFrame()
        df["date"] = env.data_date
        df["Open"] = env.data_open
        df["High"] = env.data_high
        df["Low"] = env.data_low
        df["Close"] = env.data_close
        df["Volume"] = env.data_vol
        df["Drawdown"] = [0 for i in range(self.max_look_back)] + self.drawdown_lis
        df["Marketposition"] = [0 for i in range(self.max_look_back)] + self.marketposition_list
        df["Equity"] = [self.initial_cash for i in range(self.max_look_back)] + self.value_list
        df["PnL"] = [0 for i in range(self.max_look_back)] + self.PnL_list
        df["date"] =pd.to_datetime(df["date"])
        df.set_index(["date"],inplace =True)    
        return df
    
        
            
            
    def plot_info(self, start, end):
        if start < 0 or end < 0:
            raise ValueError("start and end must be positive")    
        if start > end:
            raise ValueError("start must be less than end")    
            
        self.cal_info(start, end) 
        
        font = {'weight': 'normal', 'size': 18}
    
        plt.figure(figsize = (20,16))

        plt.subplot(311)  
        plt.title('Marketposition', font)
        plt.plot(self.xs, self.open_plot)
        plt.xlabel('Date', font)
        plt.ylabel('Open price', font)
        plt.scatter(self.xs[self.long_position], self.open_plot[self.long_position], c = 'r', s = 15, label = 'long_position')
        plt.scatter(self.xs[self.short_position], self.open_plot[self.short_position], c = 'g', s = 15, label = 'short_position')
        plt.scatter(self.xs[self.neutral_position], self.open_plot[self.neutral_position], c = 'k', s = 15, label = 'neutral_position')
        plt.grid(linestyle = '-.', c='C7')
        plt.legend(fontsize = 18)
        
        plt.subplots_adjust(wspace = 0, hspace = 0.3)

        plt.subplot(312)  
        plt.title('Equity', font)
        plt.plot(self.xs, self.value_list[start:end])
        plt.xlabel('Date', font)
        plt.ylabel('Value', font)
        plt.grid(linestyle = '-.', c='C7')
        
        plt.subplots_adjust(wspace = 0, hspace = 0.3)

        plt.subplot(313)  
        plt.title('Drawdown', font)
        plt.plot(self.xs, self.drawdown_lis[start:end], c = 'r')
        plt.xlabel('Date', font)
        plt.ylabel('Drawdown', font)
        plt.grid(linestyle = '-.', c='C7')
        
        plt.show()
        plt.close()

## Optimization

In [65]:
def optimize(env, strategy, args):   
    p = 1
    for i in range(len(args)):
        if type(args[i]) == tuple or type(args[i]) == list:
            if len(args[i]) == 2:
                p = p * (args[i][1] - args[i][0])
            elif len(args[i]) == 3:
                raise ValueError('未完待續! 還要再新增處理有step的狀況')
            else:
                raise ValueError('Input must be list or tuple of [start, end, step]')
        else:
            pass

    para = np.empty((p, len(args)))

    dim1 = p 
    for j in range(len(args)):
        if type(args[j]) == tuple or type(args[j]) == list:
            para_range = (args[j][1] - args[j][0])
            p = int(p / para_range)
            for m in range(int(dim1 / (p*para_range))):
                m = m * p * para_range
                for k,l in enumerate(range(args[j][0], args[j][1])):
                    para[k*p+m : (k+1)*p+m, j] = l
        else:
            para[:, j] =  args[j]
            
    value_list = []
    t = 0
    with progressbar.ProgressBar(max_value=len(para)) as bar:
        while t < len(para): 
            
            env.reset()
            strategy.reset()
            strategy.set_para(*para[t])
            
            while env.step_counter < env.data_len:
                if len(strategy.data) < strategy.max_look_back: ##蒐集足夠的資料
                    strategy.get_data(data=env.get_state()) 
                    env.step_counter += 1
                else:
                    strategy_output = strategy.core()
                    env.step_operate(strategy_output)
                    strategy.get_data(data=env.get_state()) 
                    env.step_counter += 1   

            value_list.append(np.hstack((para[t], env.total_value)))
            t += 1
            bar.update(t)
    
    value_list_df = pd.DataFrame(value_list)
    para_columns = strategy.para_describe.copy()
    para_columns.append('Profit')
    value_list_df.columns = para_columns
    value_list_df = value_list_df.sort_values(by=['Profit'], ascending=False).reset_index(drop=True)
    
    return value_list_df

## MA test

In [66]:
class strategy_MA(object):
    def __init__(self, max_look_back):
        self.max_look_back = max_look_back
        self.reset()
        
    #清空以防萬一,雖然不清空也能運行,但清空可能未來會有用處!?一時想不到,待討論 XD    
    def reset(self):
        self.data = deque(maxlen=self.max_look_back)
        self.data_date = []
        self.data_open = []
        self.data_high = []
        self.data_low = []
        self.data_close = []
        self.data_vol = []
        self.marketposition = []
        self.openposition = []
  
    def get_data(self, data):
        self.data.append(data)
        if len(self.data) == self.max_look_back:
            self.data_date = [self.data[i][0] for i in range(self.max_look_back-1,-1,-1)]
            self.data_open = [self.data[i][1] for i in range(self.max_look_back-1,-1,-1)]
            self.data_high = [self.data[i][2] for i in range(self.max_look_back-1,-1,-1)]
            self.data_low = [self.data[i][3] for i in range(self.max_look_back-1,-1,-1)]
            self.data_close = [self.data[i][4] for i in range(self.max_look_back-1,-1,-1)]
            self.data_vol = [self.data[i][5] for i in range(self.max_look_back-1,-1,-1)]
            self.marketposition = [self.data[i][6] for i in range(self.max_look_back-1,-1,-1)]
            self.openposition = [self.data[i][7] for i in range(self.max_look_back-1,-1,-1)]
        else:
            pass
        
    
    def set_para(self, *arg):
        self.slen = int(arg[0])
        self.llen = int(arg[1])
        self.loss_limit = int(arg[2])
        self.profit_limit = int(arg[3])
        
        self.para_describe = ['slen', 'llen', 'loss_limit', 'profit_limit']
        
        if self.max_look_back < self.slen or self.max_look_back < self.llen:
            raise ValueError("max_look_back must be larger than all parameters of period")  
        if self.loss_limit > 0:
            raise ValueError("loss_limit must be negative")  
        if self.profit_limit < 0:
            raise ValueError("profit_limit must be positive") 

    def core(self):
        sma = sum([self.data_open[i] for i in range(self.slen)]) / self.slen
        lma = sum([self.data_open[i] for i in range(self.llen)]) / self.llen
        
    ##每次進場合約數,目前尚未完工,只支援進出相同口數
        contract = 1    
        
    ##停損停利要先判定,不然會有BUG,原因在於如果拔除原版self.marketposition[0] == 0: 這條,那會變成幾乎不會進到停損停利的條件內
        if self.openposition[0] < self.loss_limit or self.openposition[0] > self.profit_limit:
            if self.marketposition[0] > 0 :
                output = 'Sell'
            elif self.marketposition[0] < 0 :
                output = 'Buytocover'  
        elif self.marketposition[0] == 0: 
            if sma > lma :
                output = 'Buy'
        ## 進場這邊不能用else,不然這樣 SMA = LMA會直接判定成Sellshort
            elif sma < lma :
                output = 'Sellshort'
        ##防止 SMA = LMA的情況
            else : 
                output = 'keep'
    ## keep一定要加在最外側,不然也是容易會產生Bug(不一定會Error,可能是報表跟邏輯會搭不上)
        else:
            output = 'keep'     

    ## 加入每次下單合約張數
        return [output,contract]

In [67]:
arg = [7,15,-30000,40000]

strategy_ma = strategy_MA(max_look_back = 20)
strategy_ma.set_para(*arg)

path = 'C:\\Users\\G3229\\backtesting\\TXF1_day.csv'
env = backtest_sys(data_path = path, max_look_back = strategy_ma.max_look_back)

while env.step_counter < env.data_len:
    if len(strategy_ma.data) < strategy_ma.max_look_back:
        strategy_ma.get_data(data=env.get_state()) 
        env.step_counter += 1
    else:
        strategy_output = strategy_ma.core()[0]
        strategy_contract = strategy_ma.core()[1]
        env.step_operate(strategy_output,strategy_contract)
        strategy_ma.get_data(data=env.get_state()) 
        env.step_counter += 1

env.total_value

232200.0

In [68]:
env.Total_trade_calculation()[48:51] #交易明細(還有很多績效可以算)

,Entry_price,Entry_time,Long/Short,Cover_price,Cover_time,PnL
48,7980.0,1999/9/20,Short,7439.0,1999/9/28,107000.0
49,7439.0,1999/9/29,Short,7620.0,1999/10/2,-37400.0
50,7570.0,1999/10/4,Short,7780.0,1999/10/13,-43200.0


In [69]:
env.cal_info(0,500)
env.calculation_DataFrame()[300:502] #每筆資料對應損益與倉位
env.calculation_DataFrame()['1999-9-29':'1999-10-2']

,Open,High,Low,Close,Volume,Drawdown,Marketposition,Equity,PnL
date,,,,,,,,,
1999-09-29,7439.0,7600.0,7392.0,7589.0,3666,-30600.0,-1,1599200.0,-30600.0
1999-09-30,7539.0,7630.0,7530.0,7540.0,2448,-20800.0,-1,1609000.0,9800.0
1999-10-01,7613.0,7648.0,7555.0,7648.0,2507,-42400.0,-1,1587400.0,-21600.0
1999-10-02,7620.0,7648.0,7532.0,7599.0,2083,-37400.0,0,1592400.0,5000.0


In [ ]:
##--------------以上,下面就沒更改了---------------
##目前最大問題在於整個系統都是以next bar判定,這樣的停損停利會變很不準確,進場也限制為next bar
##後續開發方向可能是以stop & limit 加上能及時停損停利為主,還有cross over 跟 cross under的概念
##績效部分還有很多可以算
##回測準確性我還沒辦法完全斷定,但如果策略邏輯是嚴謹並且不會產生同K棒進出問題的話,基本上應該都是對的
##承上點,這系統比MC優的地方在於不會同跟K棒進出,因為每次只會五選一(Buy,Sellshort,Buytocover,Sell,keep)

### test optimization of MA

In [63]:
strategy_ma = strategy_MA(max_look_back=30)

path = 'C:\\Users\\G3229\\backtesting\\TXF1_day.csv'
env = backtest_sys(data_path = path, max_look_back = strategy_ma.max_look_back)

args = [(5,10), (16,20), -30000, 40000]
outcome = optimize(env, strategy_ma, args)

outcome

TypeError: __init__() got an unexpected keyword argument 'max_value'

## KD test 測試有step的情況 未完待續

In [11]:
class strategy_KD(object):
    def __init__(self, max_look_back):
        self.max_look_back = max_look_back
        self.data = deque(maxlen=self.max_look_back)
        
    def reset(self):
        self.data.clear()
  
    def get_data(self, data): # collect the data to build the strategy
        self.data.append(data)
        if len(self.data) == self.max_look_back:
            self.data_date = [self.data[i][0] for i in range(self.max_look_back-1,-1,-1)]
            self.data_open = [self.data[i][1] for i in range(self.max_look_back-1,-1,-1)]
            self.data_high = [self.data[i][2] for i in range(self.max_look_back-1,-1,-1)]
            self.data_low = [self.data[i][3] for i in range(self.max_look_back-1,-1,-1)]
            self.data_close = [self.data[i][4] for i in range(self.max_look_back-1,-1,-1)]
            self.data_vol = [self.data[i][5] for i in range(self.max_look_back-1,-1,-1)]
            self.marketposition = [self.data[i][6] for i in range(self.max_look_back-1,-1,-1)]
            self.openposition = [self.data[i][7] for i in range(self.max_look_back-1,-1,-1)]
        else:
            pass
        
    ################################### Build the strategy below ##################################
            
    def set_para(self, *arg):
        self.slen = int(arg[0])
        self.t = float(arg[1])
        self.loss_limit = int(arg[2])
        self.profit_limit = int(arg[3])
        
        self.para_describe = ['slen', 't', 'loss_limit', 'profit_limit']
        
        if self.max_look_back < self.slen:
            raise ValueError("max_look_back must be larger than all parameters of period")  
        if self.t > 1 or self.t < 0:
            raise ValueError("t must be between 0 and 1")  
        if self.loss_limit > 0:
            raise ValueError("loss_limit must be negative")  
        if self.profit_limit < 0:
            raise ValueError("profit_limit must be positive")    
            
    def core(self): 

        rsv = ((self.data_close[0] - min(self.data_low[:self.slen])) / (max(self.data_high[:self.slen]) - min(self.data_low[:self.slen]))) * 100
        
        try:
            self.k = t * rsv + (1-t) * self.k
            self.d = t * self.k + (1-t) * self.d
        except AttributeError: 
            self.k = 50
            self.d = 50
            self.k = t * rsv + (1-t) * self.k
            self.d = t * self.k + (1-t) * self.d 
            
        if self.marketposition[0] == 0:
            if self.k  > self.d:
                output = 'long'
            else:
                output = 'short'
        else:
            if self.openposition[0] < self.loss_limit or self.openposition[0] > self.profit_limit:
                output = 'close'
            else:
                output = 'keep'
        return output

In [ ]:
strategy_kd = strategy_KD(max_look_back=15)

path = 'D:/Documents/ipython/data/backtest_sys/TXF1_day.csv'
env = backtest_sys(data_path = path, max_look_back = strategy_ma.max_look_back)

args = [(5,10), (0.1,0.9,0.1), -30000, 40000]
outcome = optimize(env, strategy_kd, args)

outcome